In [1]:
import feedparser
import pandas as pd
from datetime import datetime

# URL del feed RSS ufficiale Eurostat
rss_url = "https://ec.europa.eu/eurostat/api/dissemination/catalogue/rss/en/statistics-update.rss"
today_str = datetime.utcnow().strftime('%Y-%m-%d')
csv_file = f"eurostat_updates_{today_str}.csv"

# Parsing feed RSS
feed = feedparser.parse(rss_url)
records = []
for entry in feed.entries:
    records.append({
        'published': entry.published,
        'title': entry.title,
        'description': entry.get('description', '').strip(),
        'link': entry.link
    })

# DataFrame e dataset_id
df = pd.DataFrame(records)
df['dataset_id'] = df['title'].str.split(' - ').str[0].str.strip()

# Riordina le colonne
df = df[['published', 'dataset_id', 'title', 'description', 'link']]
df = df.sort_values('published', ascending=False)

df['json'] = df['dataset_id'].apply(
    lambda x: f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{x}?geo=IT"
)

# Salva su CSV
df.to_csv(csv_file, index=False)

print(df[['published','dataset_id', 'description','link']].head(300).to_markdown(index=False, tablefmt="github"))

| published             | dataset_id      | description                                                                                                                                                                           | link                                                                                                    |
|-----------------------|-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|
| 2025-07-10 23:00:00.0 | LIFE_SIT        | Life situation                                                                                                                                                                        | https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/codelist/ESTAT/LIFE_SIT/1.0?format=TSV&lang=EN 